In [ ]:
import networkx as nx
import matplotlib as plt
# import pygraphviz as pv
import numpy as np

# States
HEALTH = 0
TRANSMISSION = 1
IMMUNE = 2 # immune & inactive

# parameter
p1 = 0.08 # health to transmission
p2 = 0.55 # immune to transmission
a = 0.45 # user growth rate per month
b = 0.3 # inactive rate per month

class social_network():
    def __init__(self,  alpha=0.1,beta=0.3, n = 4000,name = "ws",):
        if name == "ws":
            self.graph = nx.watts_strogatz_graph(n, k = 57, p=0.5)
        elif name == "ba":
            self.graph = nx.barabasi_albert_graph(n, m = 6)
        
        
        init_dist=np.random.choice([TRANSMISSION,IMMUNE,HEALTH],
                         size = n,p=(alpha,beta,1-alpha-beta))
        
        # list of nodes group by status
        self.hea_list = np.where(init_dist==HEALTH)[0].tolist()
        self.trans_list = np.where(init_dist==TRANSMISSION)[0].tolist()
        self.im_list = np.where(init_dist==IMMUNE)[0].tolist()
        
        for i in range(0,n):
            self.graph.node[i]['status'] = init_dist[i]


In [ ]:
G=social_network(name ="ba")
g = G.graph

In [ ]:
trans_list = G.trans_list
hea_list = G.hea_list
im_list = G.im_list
nodes = g.nodes(data = True)

origin_sta = np.array([g.node[nei]['status'] 
                        for nei in range(0,4000)])
origin_health = set(np.where(origin_sta == HEALTH)[0])
origin_imm = set(np.where(origin_sta == IMMUNE)[0])
origin_heal_list = set(copy.deepcopy(hea_list))
origin_im_list = set(copy.deepcopy(im_list))
origin_trans_list = set(copy.deepcopy(trans_list))

change_to_unhealth = set([])
change_to_imm = set([])

spreaded_all = set([])
leave_trans = set([])

In [ ]:
for nt in trans_list:

    nei_list = np.array(g.neighbors(nt))
    nei_sta = np.array([g.node[nei]['status'] 
                        for nei in nei_list])
    
    heal_nei = nei_list[np.where(nei_sta == HEALTH)[0]]
    spreaded_nei = heal_nei[np.where(np.random.uniform(0,1,(heal_nei.shape[0],)) 
                              < p1)[0]]

    if spreaded_nei.size!=0:    # if spread to nei
        trans_list.remove(nt)
        im_list.append(nt)
        leave_trans.add(nt)
        #
        change_to_unhealth.update(spreaded_nei)
        change_to_imm.add(nt)
        spreaded_all.update(spreaded_nei)
        #
        g.node[nt]['status'] = IMMUNE
        
        
        
        for sn in spreaded_nei:
            hea_list.remove(sn)
            trans_list.append(sn)
            
            
            g.node[sn]['status'] = TRANSMISSION
    elif np.random.uniform(0,1) < p2:
        im_list.append(nt)
        g.node[nt]['status'] = IMMUNE
        #
        change_to_imm.add(nt)
        leave_trans.add(nt)